In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import backend as K
from tqdm import tqdm

In [2]:
#set up data processing function
def norm(data):
    a= preprocessing.StandardScaler().fit(data)
    d=a.transform(data)
    m=a.mean_
    s=a.scale_
    return m,s
def mish(x):
        return x * K.tanh(K.softplus(x))    

def mae(x,y):
    return np.abs(x-y)
def feature_selector(model,x,norm):
    normalized_data=(np.transpose(x,(0,2,1))-norm[0])/norm[1]
    return model.predict(normalized_data,batch_size=128)
def concat_data(x1,x2,x3):
    normalized_data=(x3-summary_norm[0])/summary_norm[1]
    return np.hstack((x1,x2,normalized_data))
def re_norm(cell_feature):
    log1=[]
    log2=[]
    for i in range(len(cell_feature)):
        len_=len(cell_feature['%d'%(i)])-100
        for k in range(len_):
            for j in range(0,50,1):            
                log1.append(np.float32(k))
                log2.append(np.float32(eol_data[i]-k))
    log1=np.float32(norm(np.array(log1).reshape(-1,1)))
    log2=np.float32(norm(np.array(log2).reshape(-1,1)))
    return log1,log2
def process2predict(cell_feature):
    s_norm,rul_norm=re_norm(cell_feature)
    x_in=[]
    y_in=[]
    for i in range(len(cell_feature)):
        col1=[]
        col2=[]
        len_=len(cell_feature['%d'%(i)])-100  
        for k in range(len_):
            for j in range(0,50,1):            
                temp=cell_feature['%d'%(i)][k:(j+k+1)]
                col1.append(np.float32(np.pad(temp, ((0,50-j-1),(0,0)), 'edge')))
                col2.append(np.float32(((eol_data[i]-k))-rul_norm[0])/rul_norm[1])
                col2.append((np.float32(k)-s_norm[0])/s_norm[1])
        x_in.append(col1)
        y_in.append(col2)
    return x_in,y_in

In [3]:
#loading_data

data_dir='dataset'
model_dir='pretrained'


eol_data = np.load('%s/battery_EoL.npy'%(data_dir),allow_pickle='TRUE')
battery_id = np.load('%s/index_battery.npy'%(data_dir),allow_pickle='TRUE')
charge_data=np.load('%s/charge_data.npy'%(data_dir),allow_pickle='TRUE').tolist()
discharge_data=np.load('%s/discharge_data.npy'%(data_dir),allow_pickle='TRUE').tolist()
summary_data=np.load('%s/summary_data.npy'%(data_dir),allow_pickle='TRUE').tolist()
charge_norm=np.load('%s/charge_norm.npy'%(data_dir),allow_pickle='TRUE').tolist()
discharge_norm=np.load('%s/discharge_norm.npy'%(data_dir),allow_pickle='TRUE').tolist()
summary_norm=np.load('%s/summary_norm.npy'%(data_dir),allow_pickle='TRUE').tolist()
index=np.load('%s/index_battery.npy'%(data_dir))
predict_renorm=np.load('%s/predict_renorm.npy'%(data_dir))

In [4]:
#loading pre-trained model
feature_selector_ch=tf.keras.models.load_model('%s/feature_selector_ch.h5'%(model_dir), compile=False)
feature_selector_dis=tf.keras.models.load_model('%s/feature_selector_dis.h5'%(model_dir), compile=False,custom_objects={'mish':mish})
predictor=tf.keras.models.load_model('%s/predictor.h5'%(model_dir), compile=False,custom_objects={'mish':mish})

In [5]:
# set the prediction sample
cell_number=9 #battery id
cycle=100     #strating cycle
append=5      #attach n cycle feature 

In [6]:
#feature selector & concatenate data-driven feature and Summary feature
charge_feature=feature_selector(feature_selector_ch,
                                charge_data[cell_number][cycle:(cycle+append)],charge_norm)
discharge_feature=feature_selector(feature_selector_dis,
                                discharge_data[cell_number][cycle:(cycle+append)],discharge_norm)
cell_feature=concat_data(charge_feature,
                         discharge_feature,
                           summary_data[cell_number][cycle:(cycle+append)])    
cell_feature=np.pad(cell_feature, ((0,50-len(cell_feature)),(0,0)), 'edge').reshape(-1,50,12)


In [7]:
#Prediction & compute the error

p1=(predictor.predict(cell_feature)*predict_renorm[:,1]+predict_renorm[:,0])[0]
ans=np.array([eol_data[cell_number]-cycle,cycle])

print('Predict RUL/S is %d/%d'%(p1[0],p1[1]))
print('Ground truth RUL/S is %d/%d'%(ans[0],ans[1]))

Predict RUL/S is 611/82
Ground truth RUL/S is 609/100
